## PART I: Ensure New Image Creation Works

In [ ]:
from rime_sdk import Client

## Setup Experiment

Please enter values for the cluster, its API token, and an image you want to create.

In [ ]:
API_TOKEN = ""                        # CREATE & PASTE API_KEY.
CLUSTER_URL = ""  # PASTE DEDICATED DOMAIN OF RIME SERVICE (eg: rime.example.rbst.io).
IMAGE_NAME = "my_managed_image"                       # SELECT A UNIQUE IMAGE NAME
REQ_NAME = "pyjokes"
REQ_VERSION_SPEC="==0.6.0"

In [ ]:
rime_client = Client(CLUSTER_URL, API_TOKEN)

In [ ]:
job = rime_client.create_managed_image(IMAGE_NAME, [Client.pip_requirement(name=REQ_NAME, version_specifier=REQ_VERSION_SPEC)])

In [ ]:
job.get_status(wait_until_finish=True, verbose=True)

If that succeeded, make sure your image is created by running:

In [ ]:
print([(i['name'], i['rime_tag']) for i in rime_client.list_managed_images()])

If that succeeded, start a new stress test with your image:

In [ ]:
proj = rime_client.get_project(project_id="8560fc7b-542e-44a2-8992-b34686f1ca66")

In [ ]:
ds_config = {"connection_info": {"data_file": {"path": "s3://rime-datasets/titanic/titanic_example.csv",},}, "data_params": {"label_col": "Survived",},}
ds_id = proj.register_dataset("titanic", ds_config)

In [ ]:
model_config = {"model_path": {"path": "s3://rime-models/titanic_s3_test/titanic_example_model.py",},}
mod_id = proj.register_model("titanic", model_config)

In [ ]:
config = {"run_name": "Titanic", "data_info": {"ref_dataset_id": ds_id, "eval_dataset_id": ds_id}, "model_id": mod_id, "run_time_info": {"custom_image": {"managed_image_name": IMAGE_NAME}}}

In [ ]:
stress_job = rime_client.start_stress_test(test_run_config=config, project_id=proj.project_id)

In [ ]:
stress_job.get_status(wait_until_finish=True, verbose=True)